In [1]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

client.is_ready()

## Create a collection with no vectorizer

In [ ]:
from weaviate.classes.config import Configure, VectorDistances, Property, DataType

# Delete the collection if it already exists
# if (client.collections.exists("MyCollection")):
    # client.collections.delete("MyCollection")

client.collections.create(
    name="MyCollection",

    # vectorizer_config=, # TODO: select none vectorizer

    vector_index_config=Configure.VectorIndex.hnsw( # Optional
        # TODO: add distance_metric with cosine  select prefered distance metric
    ),

    properties=[ # defining properties (data schema) is optional
        Property(name="title", data_type=DataType.TEXT), 
        # TODO: add Property points of type INT
    ]
)

print("Successfully created collection: MyCollection.")

## Insert an object with a vector

In [ ]:
my_collection = client.collections.get("MyCollection")

# TODO:  insert vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]


Fetch objects with vectors

In [ ]:
# TODO:  fetch objects to display your vector


## Insert many objects with their vectors using batch

In [ ]:
source = [
    {
        "title": "Second Object",
        "points": 22,
        "vector": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    },
    {
        "title": "Third Object",
        "points": 33,
        "vector": [0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
    },
    {
        "title": "Fourth Object",
        "points": 44,
        "vector": [0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
    },
    {
        "title": "Fifth Object",
        "points": 55,
        "vector": [0.5, 0.5, 0, 0, 0, 0]
    },
]

In [ ]:

with my_collection.batch.dynamic() as batch:
    # TODO:  iterate through objects here and add them using batch

## Example with insert_many

In [ ]:
# sample_data = [
#    wc.DataObject(
#       properties={
#          "title": "First Object",
#          "points": 11, 
#       },
#       vector=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Second Object",
#          "points": 22,
#       },
#       vector=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Third Object",
#          "points": 33,
#       },
#       vector=[0.3, 0.1, -0.1, -0.3, -0.5, -0.7]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fourth Object",
#         "points": 44,
#       },
#       vector=[0.4, 0.41, 0.42, 0.43, 0.44, 0.45]
#    ),
#    wc.DataObject(
#       properties={
#          "title": "Fifth Object",
#          "points": 55,
#       },
#       vector=[0.5, 0.5, 0, 0, 0, 0]
#    ),
# ]

# my_collection.data.insert_many(sample_data)

## Query
Available types of queries you can run when working with vector embeddings (without modules) in **Weaviate**:

1. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object)

### nearVector Example
**First example** - Search Weaviate with a vector embedding, and return title property.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-a-vector) for more.

In [ ]:
# TODO:  search near vector [-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
# display the uuid and properties

# response = my_collection.query

for item in response.objects:
    print(item.uuid)
    print(item.properties, "\n")

**Second example** - The same search query, but this time also return `distance`, and `vector`.

In [ ]:
from weaviate.classes.query import MetadataQuery

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    include_vector=True,
    # return_metadata=,  # TODO: request distance from metadata 
    limit=2,
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance)
    print(item.vector, "\n")

**Third example** – Same vector query, but this time we will filter on "points" (which should be greater than 44). Also, let's return "title" and "points".

See [the docs](https://weaviate.io/developers/weaviate/search/filters#filter-with-one-condition) for more.

In [ ]:
from weaviate.classes.query import Filter, MetadataQuery

response = my_collection.query.near_vector(
    near_vector=[-0.012, 0.021, -0.23, -0.42, 0.5, 0.5],
    return_metadata=MetadataQuery(distance=True),
    limit=2,
    # filters= #  TODO: add filter where points is greater_than 30
)

for item in response.objects:
    print(item.properties)
    print(item.metadata.distance, "\n")

### nearObject Example

Weaviate also allows you to search for similar objects.

See [the docs](https://weaviate.io/developers/weaviate/search/similarity#search-with-an-existing-object) for more.

**Fourth example** - 
Search through `MyCollection` for similar objects, by providing an id from the previous query. 

> Note #1: The id was taken from the query above <br/>
> The generated id for you might be different.

> Note #2: The first object returned is always itself.

In [ ]:
from weaviate.classes.query import MetadataQuery

#  TODO: search for similar objects based on uuid from a previous example
response = my_collection.query.near_object

for item in response.objects:
    print(item.uuid)
    print(item.properties)
    print(item.metadata.distance, "\n")

## Close the client when done

In [ ]:
client.close()